In [1]:
import logging
import sys

from naslib.defaults.trainer import Trainer

from naslib.search_spaces import (
    DartsSearchSpace,
    SimpleCellSearchSpace,
    NasBench101SearchSpace,
    HierarchicalSearchSpace,
)
from naslib.search_spaces.nasbench101 import graph

from naslib.utils import get_dataset_api, setup_logger, utils

import numpy as np

device: cpu
device: cpu
device: cpu
device: cpu
device: cpu
device: cpu


### Defining the Dataset

In [2]:
config = utils.get_config_from_args(config_type="nas")

In [3]:
dataset_api = get_dataset_api(config.search_space, config.dataset)

Loading dataset from file... This may take a few minutes...
Loaded dataset in 4 seconds


### Encode Sampled Architectures

In [4]:
INPUT = "input"
OUTPUT = "output"
CONV3X3 = "conv3x3-bn-relu"
CONV1X1 = "conv1x1-bn-relu"
MAXPOOL3X3 = "maxpool3x3"
OPS = [CONV3X3, CONV1X1, MAXPOOL3X3]

NUM_VERTICES = 7
OP_SPOTS = NUM_VERTICES - 2
MAX_EDGES = 9

In [5]:
def sample_random_architecture(dataset_api, arch_limit=10):
        """
        This will sample a random architecture and update the edges in the
        naslib object accordingly.
        From the NASBench repository:
        one-hot adjacency matrix
        draw [0,1] for each slot in the adjacency matrix
        """
        architectures = []
        while len(architectures) < arch_limit:
            matrix = np.random.choice([0, 1], size=(NUM_VERTICES, NUM_VERTICES))
            matrix = np.triu(matrix, 1)
            ops = np.random.choice(OPS, size=NUM_VERTICES).tolist()
            ops[0] = INPUT
            ops[-1] = OUTPUT
            spec = dataset_api["api"].ModelSpec(matrix=matrix, ops=ops)
            if dataset_api["nb101_data"].is_valid(spec):
                architectures.append({"matrix": matrix, "ops": ops})
                #break
        
        return architectures
            
        #self.set_spec({"matrix": matrix, "ops": ops})

### Sampling the architectures

* Following functions are modified from: https://github.com/kalifou/fitness_landscape_analysis_NAS/blob/main/nasbench/nasbench/nasalg/landscape.py

In [6]:
sampled_architectures = sample_random_architecture(dataset_api)

In [7]:
MAX_NODES = 7
MAX_EDGES = 9
INPUT = 'input'
OUTPUT = 'output'
CONV1X1 = 'conv1x1-bn-relu'
CONV3X3 = 'conv3x3-bn-relu'
MAXPOOL3X3 = 'maxpool3x3'

In [8]:
CODING = [INPUT]
CODING = CODING + [CONV1X1 + "_" + str(i) for i in range(0, (MAX_NODES - 2))]
CODING = CODING + [CONV3X3 + "_" + str(i) for i in range(0, (MAX_NODES - 2))]
CODING = CODING + [MAXPOOL3X3 + "_" + str(i) for i in range(0, (MAX_NODES - 2))]
CODING = CODING + [OUTPUT]

In [9]:
def rename_ops(ops):
    c1x1 = 0
    c3x3 = 0
    mp3x3 = 0
    new_ops = []
    for op in ops:
        if op == CONV1X1:
            new_ops = new_ops + [op + "_" + str(c1x1)]
            c1x1 = c1x1 + 1
        elif op == CONV3X3:
            new_ops = new_ops + [op + "_" + str(c3x3)]
            c3x3 = c3x3 + 1
        elif op == MAXPOOL3X3:
            new_ops = new_ops + [op + "_" + str(mp3x3)]
            mp3x3 = mp3x3 + 1
        else:
            new_ops = new_ops + [op]
    return new_ops

In [10]:
def encode_matrix(adj_matrix, ops):
    enc_matrix = np.zeros((len(CODING), len(CODING)))
    pos = [CODING.index(op) for op in ops]
    trans = dict()
    for i, ix in enumerate(pos):
        trans[i] = ix
    i, j = np.nonzero(adj_matrix)
    ix = [trans.get(n) for n in i]
    jy = [trans.get(n) for n in j]
    for p in zip(ix, jy):
        enc_matrix[p] = 1
    encoded = enc_matrix[np.triu_indices(len(CODING), k=1)]
    return encoded.astype(int)

In [11]:
def encode_solution(solution):
    adj_matrix = solution['matrix'] # module adjacency
    ops = rename_ops(solution['ops']) # module operations
    encoded = encode_matrix(adj_matrix, ops)    
    return encoded

In [12]:
arch_1 = sampled_architectures[0]

In [13]:
print(arch_1)

{'matrix': array([[0, 1, 0, 1, 0, 1, 0],
       [0, 0, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0]]), 'ops': ['input', 'conv3x3-bn-relu', 'conv3x3-bn-relu', 'conv1x1-bn-relu', 'conv1x1-bn-relu', 'conv1x1-bn-relu', 'output']}


In [14]:
encoded_sampled_arcs = []

In [15]:
for sampled_arc in sampled_architectures:
    encoded_sampled_arcs.append(encode_solution(sampled_arc))